In [1]:
#| default_exp icao
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# ICAO

> Este módulo concentra as constantes, funções de carga, processamento, mesclagem e salvamento de dados aeronáuticos provenientes do Software Frequency Finder da ICAO

In [2]:
#| export
from datetime import datetime
from typing import Iterable
import pandas as pd
from fastcore.test import test_close

## CONSTANTES


Dados para acesso aos dados do ICAO

In [3]:
#| export
PATH_NAV = r"\\servrepds\dw$\Input\sentinela\extracao\VHF_NAV_20221108.xlsx"
PATH_COM = r"\\servrepds\dw$\Input\sentinela\extracao\VHF_COM_20221108.xlsx"
PATH_CHANNELS = r"\\servrepds\dw$\Input\sentinela\extracao\VOR_ILS_DME_Channel.xlsx"
COLS_NAV = ["Frequency", "Latitude", "Longitude", "Facility", "Location", "NS", "WE"]
COLS_COM = ["Frequency", "CoordLat", "CoordLong", "DOC", "Location", "NS", "WE"]

In [4]:
#| export
def convert_latitude(
    lat: str,  # Latitude
    hemisphere: str,  # Hemisfério: N | S
) -> float:
    """Converte a Latitude para formato decimal"""
    multiplier = 1 if hemisphere == "N" else -1
    return multiplier * (
        float(lat[:2]) + float(lat[3:5]) / 60 + float(lat[6:8]) / 3600.0
    )


def convert_longitude(
    lon: str,  # Longitude
    hemisphere: str,  # Hemisfério: W | E
) -> float:
    """Converte a longitude para formato decimal"""

    multiplier = 1 if hemisphere == "E" else -1
    return multiplier * (
        float(lon[1:3]) + float(lon[4:6]) / 60 + float(lon[7:9]) / 3600.0
    )


In [5]:
lat = "25D36'01"
long = "056D06'00"
test_close(convert_latitude(lat, 'S'), -25.60027)
test_close(convert_longitude(long, 'W'), -56.10000)

In [6]:
#|export
def _read_df(
    path: str,  # Caminho do arquivo
    usecols: Iterable[str],  # Subconjunto de colunas do arquivo
) -> pd.DataFrame:  # Dataframe formatado
    # sourcery skip: use-fstring-for-concatenation
    """Lê o DataFrame no caminho `path`, filtra as colunas `usecols` e o retorna formatado"""
    df = pd.read_excel(path, engine="openpyxl", dtype="string")[usecols]
    df.columns = COLS_NAV
    df["Latitude"] = df.apply(
        lambda x: convert_latitude(x["Latitude"], x["NS"]), axis=1
    )
    df["Longitude"] = df.apply(
        lambda x: convert_longitude(x["Longitude"], x["WE"]), axis=1
    )
    df["Description"] = "[ICAO] " + df.Facility + ", " + df.Location
    return df[["Frequency", "Latitude", "Longitude", "Description"]]


In [7]:
#| export
def map_channels(
    df: pd.DataFrame,  # DataFrame dos dados de origem
) -> pd.DataFrame:
    """Mapeia os canais contidos em `df` e adiciona os registros ILS/DME caso houver"""
    chs = pd.read_excel(PATH_CHANNELS, engine="openpyxl", dtype="string")
    for row in df[df.Description.str.contains("ILS|DME")].itertuples():
        if not (ch := chs[(chs.VOR_ILSloc == row.Frequency)]).empty:
            for i, c in enumerate(ch.values[0][2:]):
                if pd.notnull(c):
                    match i:
                        case 0:
                            freq_type = "ILS glide path"
                        case 1:
                            freq_type = "Airbone DME"
                        case 2:
                            freq_type = "Ground-based DME"
                    description = (
                        f"{row.Description.replace('ICAO' , 'DOC')} ({freq_type})"
                    )
                    df.loc[len(df)] = [c, row.Latitude, row.Longitude, description]
    return df


In [8]:
#| export    
def get_icao(path_nav: str, # Caminho para o arquivo NAV
             path_com: str, # Caminho para o arquivo COM
) -> pd.DataFrame:  # DataFrame com frequências, coordenadas e descrição das estações
    """Lê, concatena e pós-processa os arquivos do ICAO"""
    df = pd.concat(
        _read_df(p, c) for p, c in zip([path_nav, path_com], [COLS_NAV, COLS_COM])
    )

    return map_channels(df)

In [9]:
PATH_NAV = Path.cwd().parent / 'dados' / 'VHF_NAV_20221108.xlsx'
PATH_COM = Path.cwd().parent / 'dados' / 'VHF_COM_20221108.xlsx'
PATH_CHANNELS = Path.cwd().parent.parent / "dados" / "VOR_ILS_DME_Channel.xlsx"

In [10]:
get_icao(PATH_NAV, PATH_COM)

/home/melinda/micromamba/envs/db/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Frequency,Latitude,Longitude,Description
0,113.4,-9.866667,-56.100000,"[ICAO] VOR/DME, ALTA FLORESTA"
1,113.2,-3.250000,-52.250000,"[ICAO] VOR/DME, ALTAMIRA"
2,117.5,-4.183333,-69.933333,"[ICAO] VOR/DME, AMAZONICA"
3,115.4,-16.250000,-49.000000,"[ICAO] VOR/DME, ANAPOLIS"
4,112,-10.983333,-37.066667,"[ICAO] VOR/DME, ARACAJU STA. MARIA"
...,...,...,...,...
2626,1176,-20.766667,-51.550000,"[DOC] VOR/DME, URUBUPUNGA CASTILHO (Ground-bas..."
2627,1082,-12.700000,-60.083333,"[DOC] VOR/DME, VILHENA (Airbone DME)"
2628,1019,-12.700000,-60.083333,"[DOC] VOR/DME, VILHENA (Ground-based DME)"
2629,1126,-20.250000,-40.283333,"[DOC] VOR/DME, VITORIA GOIABEIRAS (Airbone DME)"
